# Cart Pole

I adapt ["Actor Critic Method" by Apoorv Nandan](https://keras.io/examples/rl/actor_critic_cartpole/) for the [OpenAI Gym CartPole-v1 task](https://gymnasium.farama.org/environments/classic_control/cart_pole/).

The Actor Critic method involves two components:

* The *actor* computes a probability for each action in the state space.
* The *critic* computes the sum of all rewards the agent expects to receive in the future.

The agent learns to select actions that maximize the rewards it expects it will receive.

In the CartPole-v1 task, the agent can take two actions: push cart to the left (0) and push cart to the right (1). An observation $(x, v, \theta, \omega)$ consists of position $x$, velocity $v$, pole angle $\theta$, and angular velocity $\omega$. The agent is awarded +1 for each step taken, since the goal is to keep the pole upright as long as possible.

First, the required libraries:

In [24]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import gymnasium as gym
import numpy as np
import tensorflow.keras as keras
from tensorflow.keras import ops, Model
from tensorflow.keras.layers import Dense, Input
import tensorflow as tf

In [25]:
DISCOUNT_FACTOR = 0.99
STEPS_PER_EPISODE = 10_000
env = gym.make('CartPole-v1')

The actor and the critic share an input and hidden layer:
![Diagram of Model](./Cart%20Pole%20Actor-Critic%20Model.svg)

In [26]:
NUM_INPUTS = 4
NUM_ACTIONS = 2
NUM_HIDDEN = 128
EPS = np.finfo(np.float32).eps

inputs = Input(shape=(NUM_INPUTS,))
common = Dense(NUM_HIDDEN, activation='relu')(inputs)
action = Dense(NUM_ACTIONS, activation='softmax')(common)
critic = Dense(1)(common)

model = Model(inputs=inputs, outputs=[action, critic])

In [28]:
optimizer = keras.optimizers.Adam(learning_rate=0.01)
# model.compile(optimizer=optimizer)
critic_loss = keras.losses.Huber()
running_reward = 0
episode_count = 0

while True:
    state, _ = env.reset()
    action_probs_history = []
    expected_return_history = []
    rewards_history = []
    episode_reward = 0
    with tf.GradientTape() as tape:
        for timestep in range(1, STEPS_PER_EPISODE):
            # env.render()

            state = tf.convert_to_tensor(state)
            state = tf.expand_dims(state, 0)

            action_probs, expected_return = model(state)
            action = np.random.choice(NUM_ACTIONS, p=np.squeeze(action_probs))
            action_probs_history.append(ops.log(action_probs[0, action]))
            expected_return_history.append(expected_return[0, 0])

            state, reward, done, _, _ = env.step(action)
            rewards_history.append(reward)
            episode_reward += reward

            if done:
                break

        running_reward = 0.05 * episode_reward + (1 - 0.05) * running_reward

        # The return at a given time step is the sum of all future
        # rewards [..., r2, r1, r0] weighted iteratively by the
        # discount factor:
        #     returns[0] = r0
        #     returns[1] = r1 + Y*r0 = r1 + Y * returns[0]
        #     returns[2] = r2 + Y*(r1 + Y*r0) = r2 + Y * returns[1]
        returns = np.zeros(len(rewards_history))
        discounted_return = 0
        for i in range(len(returns)):
            discounted_return = rewards_history[-1 - i] + DISCOUNT_FACTOR * discounted_return
            returns[-1 - i] = discounted_return

        # Normalize by computing the Z-score (x - mean) / stdev.
        returns = (returns - np.mean(returns)) / (np.std(returns) + EPS)

        actor_loss = 0.
        critic_loss_ = 0.
        for action_prob, expected, return_ in zip(action_probs_history, expected_return_history, returns):
            diff = return_ - expected
            actor_loss -= action_prob * diff
            critic_loss_ += critic_loss(
                ops.expand_dims(expected, 0),
                ops.expand_dims(return_, 0)
            )

        cost = actor_loss + critic_loss_
        grads = tape.gradient(cost, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

    episode_count += 1
    if episode_count % 10 == 0:
        print(f'Running reward at episode {episode_count}: {running_reward:.2f}')

    if running_reward > 1000:
        print(f'Solved at episode {episode_count}!')
        break

Running reward at episode 10: 10.18
Running reward at episode 20: 25.27
Running reward at episode 30: 42.34
Running reward at episode 40: 40.89
Running reward at episode 50: 40.25
Running reward at episode 60: 49.19
Running reward at episode 70: 48.30
Running reward at episode 80: 78.89
Running reward at episode 90: 90.70
Running reward at episode 100: 128.60
Running reward at episode 110: 134.02
Running reward at episode 120: 161.16
Running reward at episode 130: 170.25
Running reward at episode 140: 294.16
Running reward at episode 150: 204.87
Running reward at episode 160: 171.55
Running reward at episode 170: 346.13
Running reward at episode 180: 251.56
Running reward at episode 190: 569.21
Solved at episode 193!


Once I trained the model, the cart was able to balance the pole pretty much indefinitely:

In [ ]:
import matplotlib.pyplot as plt
os.environ['SDL_VIDEODRIVER'] = 'dummy'
from IPython.display import clear_output
from tensorflow.keras.saving import load_model

def choose_action(self, state):
    state = tf.convert_to_tensor(state)
    state = tf.expand_dims(state, 0)
    action_probs, _ = model(state)
    return np.random.choice(2, p=np.squeeze(action_probs))

env = gym.make('CartPole-v1', render_mode='rgb_array')

agent = Agent('cart_pole.keras')
state, _ = env.reset()
done = False
while not done:
    clear_output(wait=True)
    frame = env.render()
    plt.imshow(frame)
    plt.show()

    action = agent.choose_action(state)
    state, _, done, _, _ = env.step(action)

